In [3]:
import requests
import pandas as pd
import re

In [19]:
url_tpl = "https://www.wikiart.org/en/paintings-by-style/{}?json=2&layout=new&page={}&resultType=masonry"
artist = "ukiyo-e"

In [20]:
page = 1
all_paintings_df = pd.DataFrame()

while True:
    url = url_tpl.format(artist, page)
    resp = requests.get(url = url)
    paintings_list = resp.json()["Paintings"]
    if paintings_list == None:
        break
    paintings_df = pd.DataFrame(paintings_list)
    all_paintings_df = pd.concat([all_paintings_df, paintings_df])
    page = page + 1
    
all_paintings_df = all_paintings_df.reset_index(drop=True)

In [25]:
num = 0

for im_url in all_paintings_df.image:
    name = re.findall("[^/]+$", im_url)[0]
    img = requests.get(im_url).content
    with open("../images/" + artist + "_raw/" + "%04d" % num + "-" + name, 'wb') as f:
        f.write(img)
    num = num + 1